In [11]:
pip install rasterio
pip install numpy
pip install pandas
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [9]:
import rasterio
import numpy as np
import pandas as pd
from pyproj import Transformer

doyeop = ['38712', '38713', '38715', '38710', '38714', '38709', '38711']

# DEM 파일 경로 (파일명에 맞게 수정)
for num in range(len(doyeop)):
    dem_path = f"C:/Users/user/Documents/에이콘아카데미/DEM/(B080)공개DEM_{doyeop[num]}_img_2022/{doyeop[num]}.img"
    
    # DEM 열기
    with rasterio.open(dem_path) as src:
        elevation = src.read(1)  # 고도 데이터 (2D 배열)
        transform = src.transform
        crs = src.crs
        width, height = src.width, src.height
    
    # 픽셀 크기
    dx = transform[0]
    dy = -transform[4]  # y 해상도는 음수 방향
    
    # 경사도 계산 (x: 동서, y: 남북 방향)
    gy, gx = np.gradient(elevation, dy, dx)
    slope_rad = np.arctan(np.sqrt(gx**2 + gy**2))
    slope_deg = np.degrees(slope_rad)
    
    # 픽셀 위치 → 실제 좌표
    rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    xs = np.array(xs).flatten()
    ys = np.array(ys).flatten()
    
    # 좌표계 변환: EPSG:5181 → EPSG:4326 (경도, 위도)
    transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
    lons, lats = transformer.transform(xs, ys)
    
    # 고도 및 경사도 데이터 1차원화
    elevation_flat = elevation.flatten()
    slope_flat = slope_deg.flatten()
    
    # 유효한 데이터만 선택 (NaN 또는 nodata 제거)
    valid_mask = ~np.isnan(elevation_flat)
    
    # DataFrame 생성
    df = pd.DataFrame({
        "longitude": np.array(lons)[valid_mask],
        "latitude": np.array(lats)[valid_mask],
        "elevation": elevation_flat[valid_mask],
        "slope": slope_flat[valid_mask]
    })
    
    # CSV 저장
    df.to_csv(f"철원{num+2}.csv", index=False)

RasterioIOError: C:/Users/user/Documents/에이콘아카데미/DEM/(B080)공개DEM_38709_img_2022/38709.img: No such file or directory